In [8]:
SYSTEM_PROMPT = """You are an urban planner tasked with analyzing candidate sites, pollution/heat clusters, and urban heat island (UHI) zones.
Your role is to generate actionable, clear, and detailed urban planning guidelines for decision-making.
Write your response in structured bullet points so it is easy to read.

📥 Input (context provided to you will look like this):

Candidate sites with details:

Coordinates (Lat, Lon: {latitude}, {longitude})

Water access: {water proximity, wetness history}

Soil: {pH, clay %, sand %, SOC g/kg, notes}

Terrain: {HAND proxy, slope, low-lying risk}

Heat: {temperature ranges, seasonality}

Urban form: {building coverage, road density}

Pollution/Cluster analysis:

Cluster ID: {cluster_id}

Area: {size km²}

Current pollution/heat level: {z-score, relative to city typical}

Seasonality: {better/worse in monsoon/dry season, Δ value}

Sensitive sites: {schools, clinics, hospitals, elder homes}

Industrial/point-source features: {list of sources}

Urban Heat Island (UHI) hot zones:

Hot zone ID: {zone_id}

Area: {size km²}

Vulnerable groups: {children %, elderly %}

Surfaces: {impervious %, tree canopy %, NDVI}

Roofs: {total m², large roofs potential m²}

Building density: {footprint cover %, height levels}

Nearest water: {m distance}

Temperature: {day °C, night °C, seasonality, extremes}

Sensitive sites: {schools, hospitals, clinics, elder homes}

Bad air quality zones:

Air quality cluster ID: {cluster_id}

 Area: {size km²}

• People living inside: {people count}

• Current level: {level code} {comment_about_level} {z-score, relative to city typical}

• Seasonality: {seasonal variation, Δ value}

• Sensitive sites inside: {schools_count, clinics_count, hospitals_count, elder_homes_count}

• Industrial/port/point-source features inside: {source_count}

📤 Expected Output (LLM should generate):

For each site/cluster/zone, provide:

1. Suitability Assessment

Strengths (what makes this site suitable for development/greenery)

Weaknesses (risks, vulnerabilities, missing data)

Key environmental and social concerns

2. Urban Planning Recommendations

Land-use suggestions (e.g., micro-park, housing, water retention, blue-green corridor)

Infrastructure needs (roads, drainage, soil remediation, water treatment)

Heat and pollution mitigation measures (e.g., tree planting, cool roofs, water buffers)

Social considerations (protecting schools, clinics, vulnerable populations)

3. Decision Guidelines

Is this site/cluster recommended for development, conservation, or monitoring?

Priority level (High/Medium/Low)

Trade-offs (e.g., risk of flooding vs. community need)

📌 Style Instructions:

Always act as an expert urban planner.

Write in clear, structured bullet points.

Provide detailed, evidence-based reasoning.

Highlight practical actions that local governments/NGOs could implement.



(You will be given multiple zone/cluster/site details separated by numbers. In your response, give desision for each cluster in this format:
<| Decision for cluster/node {cluster_id or node_id} |>
 {input parameters for this cluster}
 #### AI interpretations : 
 {Your decision as instructed above}
<| End of decision for cluster/node {cluster_id or node_id} |>
)
"""

In [18]:
import os
from groq import Groq

def call_groq_with_system_and_user(system_prompt: str, user_prompt: str, model):
    """
    Calls Groq chat completions API with a system message and a user message,
    returns the response content and token usage.
    """
    api_key = os.getenv("GROQ_API_KEY")
    if not api_key:
        raise RuntimeError("Please set your GROQ_API_KEY environment variable")

    client = Groq(api_key=api_key)

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user",   "content": user_prompt},
    ]

    response = client.chat.completions.create(
        messages=messages,
        model=model,
    )

    # Extract response text
    content = response.choices[0].message.content

    # Extract token usage
    usage = response.usage
    prompt_tokens = usage.prompt_tokens
    completion_tokens = usage.completion_tokens
    total_tokens = usage.total_tokens

    return content, prompt_tokens, completion_tokens, total_tokens

if __name__ == "__main__":
    model = "openai/gpt-oss-20b"

    system_prompt = SYSTEM_PROMPT



    user_prompt = """================= Top UHI clusters (area-wise) =================
(z (σ): standardized vs city typical; 0 ≈ typical)

Hot zone #1 (cluster 1)
• Area: ~31.10 km²
• People living inside: n/a
• Vulnerable groups: children 23.3%, elderly 1.4%
• Density / informal proxy: n/a
• Surfaces: impervious n/a | tree canopy n/a | mean NDVI n/a
• Roof area total: 10,180,842 m² | large roofs: 2869 bldgs / 2,701,936 m² (cool-roof potential)
• Building height/density: mean levels ≈ nan | 32.7% footprint cover
• Nearest water: ~0 m → blue-corridor greening potential
• Day vs Night: day 33.1 °C | night n/a | Δ n/a
• Seasonality: pre-monsoon 31.9 °C | monsoon 33.1 °C | post-monsoon n/a
• Extreme hot periods (8-day composites above local 90th pct): 1
• Heat index: n/a
• Sensitive sites: schools:319, clinics:78, hospitals:95, elder homes:1

Hot zone #2 (cluster 0)
• Area: ~4.83 km²
• People living inside: n/a
• Vulnerable groups: children 23.1%, elderly 1.6%
• Density / informal proxy: n/a
• Surfaces: impervious n/a | tree canopy n/a | mean NDVI n/a
• Roof area total: 970,975 m² | large roofs: 190 bldgs / 412,518 m² (cool-roof potential)
• Building height/density: mean levels ≈ nan | 20.1% footprint cover
• Nearest water: ~0 m → blue-corridor greening potential
• Day vs Night: day 33.6 °C | night n/a | Δ n/a
• Seasonality: pre-monsoon 31.0 °C | monsoon 33.6 °C | post-monsoon n/a
• Extreme hot periods (8-day composites above local 90th pct): 2
• Heat index: n/a
• Sensitive sites: schools:1, clinics:0, hospitals:0, elder homes:0
"""
    # model = "llama3-70b-8192"  # or whichever model you want
    

    result, prompt_tokens, completion_tokens, total_tokens = call_groq_with_system_and_user(
        system_prompt, user_prompt, model
    )

    print("Groq response:", result)
    print(f"Input tokens (prompt): {prompt_tokens}")
    print(f"Output tokens (completion): {completion_tokens}")
    print(f"Total tokens: {total_tokens}")


Groq response: <| Decision for cluster/node 1 |>
**Cluster 1 (Hot zone #1)**  
• Area: ~31.10 km²  
• Vulnerable groups: children 23.3 %, elderly 1.4 %  
• Building footprint cover: 32.7 %  
• Total roof area: 10,180,842 m² (large‑roof potential: 2,701,936 m²)  
• Nearest water: ~0 m → potential blue‑corridor  
• Daytime temperature: 33.1 °C (pre‑monsoon 31.9 °C, monsoon 33.1 °C)  
• Extreme hot periods: 1 (8‑day > 90th pct)  
• Sensitive sites: 319 schools, 78 clinics, 95 hospitals, 1 elder home  

#### AI interpretations  

**1. Suitability Assessment**  
- **Strengths**  
  - Extremely large roof area → high potential for cool‑roof retrofit or vegetated roofing.  
  - Zero distance to water → opportunity to create a blue‑green corridor that mitigates heat and provides storm‑water detention.  
  - High child and school density → community benefit from greening and cooling initiatives.  
- **Weaknesses / Risks**  
  - Very high population density implied by many schools/hospitals → li

In [23]:
import re
def parse_llm_response(text) -> dict:
    """
    Extracts cluster-wise decisions from the given text.
    
    Args:
        text (str): The full LLM response containing cluster decisions.
    
    Returns:
        dict: { cluster_id: decision_text }
    """
    cluster_pattern = re.compile(r"<\|\s*Decision for cluster/node (\d+)\s*\|>(.*?)<\|\s*End of decision for cluster/node \1\s*\|>", re.S)
    
    decisions = {}
    for match in cluster_pattern.finditer(text):
        cluster_id = int(match.group(1))
        decision_text = match.group(2).strip()
        decisions[cluster_id] = decision_text
    
    return decisions

print(parse_llm_response(result)[0])

**Cluster 0 (Hot zone #2)**  
• Area: ~4.83 km²  
• Vulnerable groups: children 23.1 %, elderly 1.6 %  
• Building footprint cover: 20.1 %  
• Total roof area: 970,975 m² (large‑roof potential: 412,518 m²)  
• Nearest water: ~0 m → potential blue‑corridor  
• Daytime temperature: 33.6 °C (pre‑monsoon 31.0 °C, monsoon 33.6 °C)  
• Extreme hot periods: 2 (8‑day > 90th pct)  
• Sensitive sites: 1 school, 0 clinics/hospitals  

#### AI interpretations  

**1. Suitability Assessment**  
- **Strengths**  
  - Smaller, more manageable area with lower building density (20.1 %) → easier to create open spaces and green corridors.  
  - Significant cool‑roof potential (≈ 40 % of total roof area).  
  - Near water source → opportunity for blue‑green integration.  
  - Limited sensitive sites → lower immediate social risk.  
- **Weaknesses / Risks**  
  - Still high daytime temperatures (~33.6 °C) and two extreme hot periods → potential heat stress for children in the single school.  
  - Lack of d

### Green Access


In [2]:
import os
from groq import Groq

def call_groq_with_system_and_user(system_prompt: str, user_prompt: str, model: str = "llama3-70b-8192"):
    """
    Calls Groq chat completions API with a system message and a user message,
    returns the response content and token usage.
    """
    api_key = os.getenv("GROQ_API_KEY")
    if not api_key:
        raise RuntimeError("Please set your GROQ_API_KEY environment variable")

    client = Groq(api_key=api_key)

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user",   "content": user_prompt},
    ]

    response = client.chat.completions.create(
        messages=messages,
        model=model,
    )

    # Extract response text
    content = response.choices[0].message.content

    # Extract token usage
    usage = response.usage
    prompt_tokens = usage.prompt_tokens
    completion_tokens = usage.completion_tokens
    total_tokens = usage.total_tokens

    return content, prompt_tokens, completion_tokens, total_tokens

if __name__ == "__main__":
    model = "openai/gpt-oss-20b"

    system_prompt = SYSTEM_PROMPT



    user_prompt = """================= Candidate Site Context =================

📍 Candidate #1  (Lat, Lon: 23.864477, 90.557852)
  Water: nearest mapped water is ~262 m (~3.4 min walk). Near water (100–500 m). — ~262 m (~3.4 min walk).
  Water presence (satellite history ≤150 m): mean 1.2826086956521743, max 2.0. Very rarely wet — area is usually dry.
  Soil (0–5 cm): pH=None (src: None), clay%=None, sand%=None, SOC g/kg=None.
  ↳ Soil pH unknown here.  |  Soil texture unknown.
  Terrain: HAND-proxy None m; slope ~None°. Low-lying risk unknown
  Heat (Apr–Jun): MODIS≈28.974218750000034°C. Heat: mild (~28.974218750000034°C).
  Urban form: building cover ~n/a% & roads ~2.7 km/km² → sparse road network

📍 Candidate #2  (Lat, Lon: 23.762769, 90.507250)
  Water: nearest mapped water is ~45 m (~0.6 min walk). Very close to water (<100 m). — ~45 m (~0.6 min walk).
  Water presence (satellite history ≤150 m): mean 62.556630971993435, max 74.0. Frequently/permanently wet — likely near river/pond or flood-prone.
  Soil (0–5 cm): pH=None (src: None), clay%=None, sand%=None, SOC g/kg=None.
  ↳ Soil pH unknown here.  |  Soil texture unknown.
  Terrain: HAND-proxy None m; slope ~None°. Low-lying risk unknown • historic water nearby
  Heat (Apr–Jun): MODIS≈30.05643835616443°C. Heat: mild (~30.05643835616443°C).
  Urban form: building cover ~1.6% & roads ~5.3 km/km² → very low building coverage; moderate road network

📍 Candidate #3  (Lat, Lon: 23.724885, 90.673295)
  Water: nearest mapped water is ~837 m (~10.7 min walk). Far from water (>500 m). — ~837 m (~10.7 min walk).
  Water presence (satellite history ≤150 m): mean 5.761538461538461, max 9.0. Occasionally wet — may pond during heavy rain.
  Soil (0–5 cm): pH=None (src: None), clay%=None, sand%=None, SOC g/kg=None.
  ↳ Soil pH unknown here.  |  Soil texture unknown.
  Terrain: HAND-proxy None m; slope ~None°. Low-lying risk unknown
  Heat (Apr–Jun): MODIS≈28.635791139240542°C. Heat: mild (~28.635791139240542°C).
  Urban form: building cover ~n/a% & roads ~4.3 km/km² → sparse road network

📍 Candidate #4  (Lat, Lon: 23.734804, 90.650293)
  Water: nearest mapped water is ~1,554 m (~19.9 min walk). Far from water (>500 m). — ~1,554 m (~19.9 min walk).
  Water presence (satellite history ≤150 m): mean 7.3151515151515145, max 9.0. Occasionally wet — may pond during heavy rain.
  Soil (0–5 cm): pH=None (src: None), clay%=None, sand%=None, SOC g/kg=None.
  ↳ Soil pH unknown here.  |  Soil texture unknown.
  Terrain: HAND-proxy None m; slope ~None°. Low-lying risk unknown
  Heat (Apr–Jun): MODIS≈28.38299342105266°C. Heat: mild (~28.38299342105266°C).
  Urban form: building cover ~n/a% & roads ~2.7 km/km² → sparse road network

📍 Candidate #5  (Lat, Lon: 23.744468, 90.685705)
  Water: nearest mapped water is ~456 m (~5.9 min walk). Near water (100–500 m). — ~456 m (~5.9 min walk).
  Water presence (satellite history ≤150 m): mean 9.884120171673823, max 14.0. Occasionally wet — may pond during heavy rain.
  Soil (0–5 cm): pH=None (src: None), clay%=None, sand%=None, SOC g/kg=None.
  ↳ Soil pH unknown here.  |  Soil texture unknown.
  Terrain: HAND-proxy None m; slope ~None°. Low-lying risk unknown
  Heat (Apr–Jun): MODIS≈28.512724358974374°C. Heat: mild (~28.512724358974374°C).
  Urban form: building cover ~0.0% & roads ~5.2 km/km² → very low building coverage; moderate road network

📍 Candidate #6  (Lat, Lon: 23.727460, 90.554811)
  Water: nearest mapped water is ~1,155 m (~14.8 min walk). Far from water (>500 m). — ~1,155 m (~14.8 min walk).
  Water presence (satellite history ≤150 m): mean 0.6666666666666666, max 1.0. Very rarely wet — area is usually dry.
  Soil (0–5 cm): pH=None (src: None), clay%=None, sand%=None, SOC g/kg=None.
  ↳ Soil pH unknown here.  |  Soil texture unknown.
  Terrain: HAND-proxy None m; slope ~None°. Low-lying risk unknown
  Heat (Apr–Jun): MODIS≈28.676666666666733°C. Heat: mild (~28.676666666666733°C).
  Urban form: building cover ~n/a% & roads ~4.3 km/km² → sparse road network

📍 Candidate #7  (Lat, Lon: 23.854673, 90.547434)
  Water: nearest mapped water is ~201 m (~2.6 min walk). Near water (100–500 m). — ~201 m (~2.6 min walk).
  Water presence (satellite history ≤150 m): mean 0.21739130434782608, max 0.0. Very rarely wet — area is usually dry.
  Soil (0–5 cm): pH=None (src: None), clay%=None, sand%=None, SOC g/kg=None.
  ↳ Soil pH unknown here.  |  Soil texture unknown.
  Terrain: HAND-proxy None m; slope ~None°. Low-lying risk unknown
  Heat (Apr–Jun): MODIS≈29.75708675799093°C. Heat: mild (~29.75708675799093°C).
  Urban form: building cover ~0.9% & roads ~3.9 km/km² → very low building coverage; sparse road network

📍 Candidate #8  (Lat, Lon: 23.599064, 90.526469)
  Water: nearest mapped water is ~182 m (~2.3 min walk). Near water (100–500 m). — ~182 m (~2.3 min walk).
  Water presence (satellite history ≤150 m): mean None, max None. No satellite evidence of open water nearby (or data unavailable).
  Soil (0–5 cm): pH=None (src: None), clay%=None, sand%=None, SOC g/kg=None.
  ↳ Soil pH unknown here.  |  Soil texture unknown.
  Terrain: HAND-proxy None m; slope ~None°. Low-lying risk unknown
  Heat (Apr–Jun): MODIS≈28.836756680731376°C. Heat: mild (~28.836756680731376°C).
  Urban form: building cover ~5.1% & roads ~9.5 km/km² → low building coverage; moderate road network

📍 Candidate #9  (Lat, Lon: 23.879706, 90.594060)
  Water: nearest mapped water is ~556 m (~7.1 min walk). Far from water (>500 m). — ~556 m (~7.1 min walk).
  Water presence (satellite history ≤150 m): mean None, max None. No satellite evidence of open water nearby (or data unavailable).
  Soil (0–5 cm): pH=None (src: None), clay%=None, sand%=None, SOC g/kg=None.
  ↳ Soil pH unknown here.  |  Soil texture unknown.
  Terrain: HAND-proxy None m; slope ~None°. Low-lying risk unknown
  Heat (Apr–Jun): MODIS≈29.09538461538463°C. Heat: mild (~29.09538461538463°C).
  Urban form: building cover ~n/a% & roads ~3.8 km/km² → sparse road network

📍 Candidate #10  (Lat, Lon: 23.864498, 90.577473)
  Water: nearest mapped water is ~1,108 m (~14.2 min walk). Far from water (>500 m). — ~1,108 m (~14.2 min walk).
  Water presence (satellite history ≤150 m): mean 6.231350330500474, max 10.0. Occasionally wet — may pond during heavy rain.
  Soil (0–5 cm): pH=None (src: None), clay%=None, sand%=None, SOC g/kg=None.
  ↳ Soil pH unknown here.  |  Soil texture unknown.
  Terrain: HAND-proxy None m; slope ~None°. Low-lying risk unknown
  Heat (Apr–Jun): MODIS≈29.31578571428575°C. Heat: mild (~29.31578571428575°C).
  Urban form: building cover ~n/a% & roads ~2.6 km/km² → sparse road network

📍 Candidate #11  (Lat, Lon: 23.591594, 90.511009)
  Water: nearest mapped water is ~82 m (~1.0 min walk). Very close to water (<100 m). — ~82 m (~1.0 min walk).
  Water presence (satellite history ≤150 m): mean 52.24524564183834, max 80.0. Frequently/permanently wet — likely near river/pond or flood-prone.
  Soil (0–5 cm): pH=None (src: None), clay%=None, sand%=None, SOC g/kg=None.
  ↳ Soil pH unknown here.  |  Soil texture unknown.
  Terrain: HAND-proxy None m; slope ~None°. Low-lying risk unknown • historic water nearby
  Heat (Apr–Jun): MODIS≈29.12262301587305°C. Heat: mild (~29.12262301587305°C).
  Urban form: building cover ~8.6% & roads ~11.4 km/km² → low building coverage; moderate road network

📍 Candidate #12  (Lat, Lon: 23.869369, 90.585840)
  Water: nearest mapped water is ~979 m (~12.5 min walk). Far from water (>500 m). — ~979 m (~12.5 min walk).
  Water presence (satellite history ≤150 m): mean None, max None. No satellite evidence of open water nearby (or data unavailable).
  Soil (0–5 cm): pH=None (src: None), clay%=None, sand%=None, SOC g/kg=None.
  ↳ Soil pH unknown here.  |  Soil texture unknown.
  Terrain: HAND-proxy None m; slope ~None°. Low-lying risk unknown
  Heat (Apr–Jun): MODIS≈29.127857142857184°C. Heat: mild (~29.127857142857184°C).
  Urban form: building cover ~n/a% & roads ~4.2 km/km² → sparse road network

📍 Candidate #13  (Lat, Lon: 23.618994, 90.564455)
  Water: nearest mapped water is ~150 m (~1.9 min walk). Near water (100–500 m). — ~150 m (~1.9 min walk).
  Water presence (satellite history ≤150 m): mean 2.3062015503875966, max 4.0. Very rarely wet — area is usually dry.
  Soil (0–5 cm): pH=None (src: None), clay%=None, sand%=None, SOC g/kg=None.
  ↳ Soil pH unknown here.  |  Soil texture unknown.
  Terrain: HAND-proxy None m; slope ~None°. Low-lying risk unknown
  Heat (Apr–Jun): MODIS≈27.95285714285717°C. Heat: mild (~27.95285714285717°C).
  Urban form: building cover ~2.1% & roads ~3.2 km/km² → very low building coverage; sparse road network

📍 Candidate #14  (Lat, Lon: 23.882651, 90.582902)
  Water: nearest mapped water is ~186 m (~2.4 min walk). Near water (100–500 m). — ~186 m (~2.4 min walk).
  Water presence (satellite history ≤150 m): mean 1.0, max 1.0. Very rarely wet — area is usually dry.
  Soil (0–5 cm): pH=None (src: None), clay%=None, sand%=None, SOC g/kg=None.
  ↳ Soil pH unknown here.  |  Soil texture unknown.
  Terrain: HAND-proxy None m; slope ~None°. Low-lying risk unknown
  Heat (Apr–Jun): MODIS≈28.78000000000003°C. Heat: mild (~28.78000000000003°C).
  Urban form: building cover ~n/a% & roads ~4.7 km/km² → sparse road network

📍 Candidate #15  (Lat, Lon: 23.698589, 90.511298)
  Water: nearest mapped water is ~49 m (~0.6 min walk). Very close to water (<100 m). — ~49 m (~0.6 min walk).
  Water presence (satellite history ≤150 m): mean 1.3571428571428572, max 2.0. Very rarely wet — area is usually dry.
  Soil (0–5 cm): pH=None (src: None), clay%=None, sand%=None, SOC g/kg=None.
  ↳ Soil pH unknown here.  |  Soil texture unknown.
  Terrain: HAND-proxy None m; slope ~None°. Low-lying risk unknown
  Heat (Apr–Jun): MODIS≈31.514962962963025°C. Heat: mild (~31.514962962963025°C).
  Urban form: building cover ~22.7% & roads ~28.7 km/km² → moderate building coverage; very dense road network

📍 Candidate #16  (Lat, Lon: 23.819471, 90.547608)
  Water: nearest mapped water is ~95 m (~1.2 min walk). Very close to water (<100 m). — ~95 m (~1.2 min walk).
  Water presence (satellite history ≤150 m): mean 61.341666666666654, max 77.0. Frequently/permanently wet — likely near river/pond or flood-prone.
  Soil (0–5 cm): pH=None (src: None), clay%=None, sand%=None, SOC g/kg=None.
  ↳ Soil pH unknown here.  |  Soil texture unknown.
  Terrain: HAND-proxy None m; slope ~None°. Low-lying risk unknown • historic water nearby
  Heat (Apr–Jun): MODIS≈29.498024691358072°C. Heat: mild (~29.498024691358072°C).
  Urban form: building cover ~1.7% & roads ~4.8 km/km² → very low building coverage; sparse road network

📍 Candidate #17  (Lat, Lon: 23.840951, 90.599897)
  Water: nearest mapped water is ~597 m (~7.7 min walk). Far from water (>500 m). — ~597 m (~7.7 min walk).
  Water presence (satellite history ≤150 m): mean 8.710909090909093, max 29.0. Occasionally wet — may pond during heavy rain.
  Soil (0–5 cm): pH=None (src: None), clay%=None, sand%=None, SOC g/kg=None.
  ↳ Soil pH unknown here.  |  Soil texture unknown.
  Terrain: HAND-proxy None m; slope ~None°. Low-lying risk unknown
  Heat (Apr–Jun): MODIS≈30.009265822784847°C. Heat: mild (~30.009265822784847°C).
  Urban form: building cover ~n/a% & roads ~1.7 km/km² → sparse road network

📍 Candidate #18  (Lat, Lon: 23.619509, 90.557210)
  Water: nearest mapped water is ~72 m (~0.9 min walk). Very close to water (<100 m). — ~72 m (~0.9 min walk).
  Water presence (satellite history ≤150 m): mean 19.677567941040998, max 40.0. Occasionally wet — may pond during heavy rain.
  Soil (0–5 cm): pH=None (src: None), clay%=None, sand%=None, SOC g/kg=None.
  ↳ Soil pH unknown here.  |  Soil texture unknown.
  Terrain: HAND-proxy None m; slope ~None°. Low-lying risk unknown
  Heat (Apr–Jun): MODIS≈28.07130511463846°C. Heat: mild (~28.07130511463846°C).
  Urban form: building cover ~2.8% & roads ~4.3 km/km² → very low building coverage; sparse road network

📍 Candidate #19  (Lat, Lon: 23.796883, 90.529181)
  Water: nearest mapped water is ~52 m (~0.7 min walk). Very close to water (<100 m). — ~52 m (~0.7 min walk).
  Water presence (satellite history ≤150 m): mean 70.36315663228595, max 80.0. Frequently/permanently wet — likely near river/pond or flood-prone.
  Soil (0–5 cm): pH=None (src: None), clay%=None, sand%=None, SOC g/kg=None.
  ↳ Soil pH unknown here.  |  Soil texture unknown.
  Terrain: HAND-proxy None m; slope ~None°. Low-lying risk unknown • historic water nearby
  Heat (Apr–Jun): MODIS≈29.47616108171665°C. Heat: mild (~29.47616108171665°C).
  Urban form: building cover ~0.1% & roads ~9.4 km/km² → very low building coverage; moderate road network

📍 Candidate #20  (Lat, Lon: 23.814329, 90.566801)
  Water: nearest mapped water is ~1,374 m (~17.6 min walk). Far from water (>500 m). — ~1,374 m (~17.6 min walk).
  Water presence (satellite history ≤150 m): mean 9.084269662921349, max 13.0. Occasionally wet — may pond during heavy rain.
  Soil (0–5 cm): pH=None (src: None), clay%=None, sand%=None, SOC g/kg=None.
  ↳ Soil pH unknown here.  |  Soil texture unknown.
  Terrain: HAND-proxy None m; slope ~None°. Low-lying risk unknown
  Heat (Apr–Jun): MODIS≈28.892968036529716°C. Heat: mild (~28.892968036529716°C).
  Urban form: building cover ~0.1% & roads ~3.7 km/km² → very low building coverage; sparse road network
"""
    # model = "llama3-70b-8192"  # or whichever model you want
    

    result, prompt_tokens, completion_tokens, total_tokens = call_groq_with_system_and_user(
        system_prompt, user_prompt, model
    )

    print("Groq response:", result)
    print(f"Input tokens (prompt): {prompt_tokens}")
    print(f"Output tokens (completion): {completion_tokens}")
    print(f"Total tokens: {total_tokens}")


Groq response: <| Decision for cluster/node 1 |>
- **Suitability Assessment**
  - **Strengths**:  
    - Moderate water proximity (~262 m) – potential for rainwater harvesting or micro‑green corridors.  
    - Mild heat (≈29 °C Apr–Jun) – low UHI risk.  
    - Sparse built environment (low road density, very low building cover) – space for open land uses.  
  - **Weaknesses**:  
    - No soil data (pH, texture, SOC) – unknown fertility and drainage.  
    - HAND proxy and slope not available – flood risk uncertain.  
  - **Key Environmental & Social Concerns**:  
    - Uncertain flood risk due to lack of HAND data.  
    - Potential vulnerability to future development pressures.

- **Urban Planning Recommendations**
  - Conduct detailed soil assessment (pH, texture, SOC) and topographic mapping to gauge flood risk.  
  - Design low‑impact development: community garden, playground, or stormwater retention pond.  
  - Incorporate tree planting along the 262 m corridor to enhance shade an

In [8]:
# map_fixed.py
import folium
import json
import branca

points = [
    {"lat": 23.8103, "lon": 90.4125, "name": "A", "desc": "Alpha cluster item, detail text."},
    {"lat": 23.8120, "lon": 90.4130, "name": "B", "desc": "Bravo detail text."},
    {"lat": 23.8110, "lon": 90.4140, "name": "C", "desc": "Charlie detail text."},
    {"lat": 23.8200, "lon": 90.4100, "name": "D", "desc": "Delta detail text."},
    {"lat": 23.8210, "lon": 90.4115, "name": "E", "desc": "Echo detail text."},
]

features = []
for i, p in enumerate(points):
    features.append({
        "type": "Feature",
        "properties": {"id": f"pt{i}", "name": p["name"], "desc": p["desc"]},
        "geometry": {"type": "Point", "coordinates": [p["lon"], p["lat"]]}
    })
geojson = {"type": "FeatureCollection", "features": features}
geojson_str = json.dumps(geojson)

m = folium.Map(location=[23.811, 90.412], zoom_start=13, tiles="OpenStreetMap")
# optional example marker created by python (keeps working)
folium.Marker([23.807, 90.408], popup="Click cluster to open side panel").add_to(m)

map_var = m.get_name()  # IMPORTANT: use this JS var name in injected JS

template = """
<!-- Load MarkerCluster plugin (CSS + JS from CDN) -->
<link rel="stylesheet" href="https://unpkg.com/leaflet.markercluster/dist/MarkerCluster.css"/>
<link rel="stylesheet" href="https://unpkg.com/leaflet.markercluster/dist/MarkerCluster.Default.css"/>
<script src="https://unpkg.com/leaflet.markercluster/dist/leaflet.markercluster-src.js"></script>

<style>
#cluster-side-panel {
  position: fixed;
  right: 12px;
  top: 60px;
  width: 320px;
  max-height: 70vh;
  overflow: auto;
  background: white;
  border: 1px solid #ccc;
  box-shadow: 0 8px 18px rgba(0,0,0,0.2);
  padding: 12px;
  z-index: 99999;
  display: none;
  border-radius: 8px;
}
#cluster-side-panel h3 { margin: 0 0 8px 0; font-size: 16px; }
#cluster-side-panel .close-btn { float: right; cursor: pointer; font-weight: bold; margin-left: 8px; }
.cluster-item { padding: 6px 6px; border-bottom: 1px solid #eee; }
.cluster-item:last-child { border-bottom: none; }
.cluster-item .title { font-weight: 600; }
.cluster-item .desc { font-size: 13px; color: #333; }
</style>

<div id="cluster-side-panel">
  <div style="display:flex; align-items:center; margin-bottom:6px;">
    <h3 id="panel-title">Cluster details</h3>
    <div class="close-btn" id="panel-close" title="Close">✕</div>
  </div>
  <div id="panel-content"></div>
</div>

<script>
(function(){
  // injected placeholders replaced below by Python
  var data = __GEOJSON__;
  var map = __MAPVAR__;

  // wait for folium's map to be ready
  map.whenReady(function() {
    var markerCluster = L.markerClusterGroup();
    var allMarkers = [];
    var selectedMarkers = [];

    data.features.forEach(function(feat, idx) {
      var coords = feat.geometry.coordinates;
      var props = feat.properties || {};
      var mkr = L.marker([coords[1], coords[0]]);
      mkr.myProps = props;
      mkr.bindTooltip(props.name || ("item " + idx), {permanent: false, direction: 'top'});
      mkr.on('click', function(e) {
        showPanelForMarkers([this], "Marker: " + (this.myProps.name||""));
        setSelectedMarkers([this]);
      });
      markerCluster.addLayer(mkr);
      allMarkers.push(mkr);
    });

    map.addLayer(markerCluster);

    markerCluster.on('clusterclick', function (a) {
      var childMarkers = a.layer.getAllChildMarkers();
      showPanelForMarkers(childMarkers, "Cluster (" + childMarkers.length + " items)");
      setSelectedMarkers(childMarkers);
      // comment out the next line if you DON'T want to zoom on cluster click
      // a.layer.zoomToBounds();
    });

    function showPanelForMarkers(markersArray, titleText) {
      var panel = document.getElementById('cluster-side-panel');
      var content = document.getElementById('panel-content');
      var title = document.getElementById('panel-title');

      title.innerText = titleText || "Cluster details";
      content.innerHTML = '';

      markersArray.forEach(function(m, idx) {
        var p = m.myProps || {};
        var div = document.createElement('div');
        div.className = 'cluster-item';
        div.innerHTML = '<div class="title">' + (p.name || ('item ' + (idx+1))) + '</div>' +
                        '<div class="desc">' + (p.desc || '') + '</div>';
        div.style.cursor = 'pointer';
        div.onclick = function() {
          map.setView(m.getLatLng(), Math.max(map.getZoom(), 15));
          m.openTooltip();
          setTimeout(function(){ m.closeTooltip(); }, 2000);
        };
        content.appendChild(div);
      });

      panel.style.display = 'block';
    }

    function setSelectedMarkers(markersArray) {
      allMarkers.forEach(function(m) { m.setOpacity(1.0); });
      var selectedSet = new Set(markersArray);
      allMarkers.forEach(function(m) {
        if (!selectedSet.has(m)) {
          m.setOpacity(0.35);
        } else {
          m.setOpacity(1.0);
          if (m._icon) { m._icon.style.transform += " translateZ(0)"; }
        }
      });
      selectedMarkers = markersArray.slice();
    }

    document.getElementById('panel-close').onclick = function() {
      document.getElementById('cluster-side-panel').style.display = 'none';
      allMarkers.forEach(function(m) { m.setOpacity(1.0); });
      selectedMarkers = [];
    };
  }); // whenReady
})();
</script>
"""

# inject the geojson and map variable name
template = template.replace("__GEOJSON__", geojson_str).replace("__MAPVAR__", map_var)

element = branca.element.Element(template)
m.get_root().html.add_child(element)

m.save("map_fixed.html")
print("Saved map_fixed.html — open it in your browser.")



Saved map_fixed.html — open it in your browser.


In [ ]:
import folium
from folium import plugins

# Location data with detailed descriptions
locations = [
    {
        "coords": [40.7589, -73.9851],
        "title": "Times Square - The Crossroads of the World",
        "location": "New York City, NY",
        "category": "Tourist Destination",
        "content": """
        <div style="max-width: 350px; font-family: Arial, sans-serif; line-height: 1.6;">
            <div style="background: #e74c3c; color: white; padding: 3px 8px; border-radius: 12px; 
                        font-size: 12px; display: inline-block; margin-bottom: 10px;">Tourist Destination</div>
            
            <h3 style="color: #2c3e50; margin-top: 0; border-bottom: 2px solid #3498db; padding-bottom: 10px;">
                The Vibrant Heart of Manhattan
            </h3>
            
            <p>Times Square stands as one of the world's most visited tourist destinations, attracting over 
            50 million visitors annually. This bustling commercial intersection has evolved from a seedy area 
            in the 1970s to become the gleaming symbol of New York City's renaissance.</p>
            
            <p>The area gets its name from The New York Times, which moved its headquarters here in 1904. 
            What started as Longacre Square was transformed into Times Square, and with it came the tradition 
            of the New Year's Eve Ball Drop that has captivated millions worldwide for over a century.</p>
            
            <p>Today, Times Square is defined by its towering digital billboards, Broadway theaters, and 
            constant flow of pedestrians from every corner of the globe. The electronic displays consume 
            enough electricity to power a small city, yet they create an atmosphere unlike anywhere else on Earth.</p>
            
            <p>The economic impact of Times Square extends far beyond its physical boundaries. The area generates 
            billions in economic activity annually, supporting countless jobs in entertainment, retail, hospitality, 
            and media. Major corporations pay millions for prime advertising space.</p>
            
            <p>From a cultural perspective, Times Square represents the democratization of urban space. Here, 
            street performers share the same stage as multi-million dollar productions, and tourists from rural 
            communities experience the same sensory overload as seasoned city dwellers.</p>
        </div>
        """
    },
    {
        "coords": [40.6892, -74.0445],
        "title": "Statue of Liberty - Symbol of Freedom",
        "location": "Liberty Island, NY",
        "category": "Historic Monument",
        "content": """
        <div style="max-width: 350px; font-family: Arial, sans-serif; line-height: 1.6;">
            <div style="background: #e74c3c; color: white; padding: 3px 8px; border-radius: 12px; 
                        font-size: 12px; display: inline-block; margin-bottom: 10px;">Historic Monument</div>
            
            <h3 style="color: #2c3e50; margin-top: 0; border-bottom: 2px solid #3498db; padding-bottom: 10px;">
                Lady Liberty: A Gift That Became an Icon
            </h3>
            
            <p>The Statue of Liberty, officially known as "Liberty Enlightening the World," stands as perhaps 
            the most recognizable symbol of freedom and democracy in the modern world. This colossal neoclassical 
            sculpture was a gift from France to the United States in 1886.</p>
            
            <p>Designed by French sculptor Frédéric Auguste Bartholdi and engineered by Gustave Eiffel, the statue 
            represents Libertas, the Roman goddess of liberty. She holds a torch above her head with her right hand, 
            while her left hand carries a tablet inscribed with the date of the American Declaration of Independence.</p>
            
            <p>The statue's construction was a marvel of 19th-century engineering. Standing 305 feet tall from base 
            to torch, Lady Liberty was the tallest iron structure of its time. The copper exterior, which has developed 
            its distinctive green patina over the decades, was revolutionary in its construction technique.</p>
            
            <p>For millions of immigrants arriving in New York Harbor between 1886 and 1924, the Statue of Liberty 
            was the first glimpse of their new homeland. The famous poem "The New Colossus" by Emma Lazarus captures 
            this sentiment with its immortal words about welcoming the tired and poor.</p>
            
            <p>During both World Wars, the statue took on additional significance as a symbol of the democratic values 
            that Allied forces were fighting to protect. Today, over 4 million people visit Liberty Island annually, 
            making it one of the most popular tourist destinations in the United States.</p>
        </div>
        """
    },
    {
        "coords": [40.7505, -73.9934],
        "title": "Empire State Building - Art Deco Marvel",
        "location": "Midtown Manhattan, NY",
        "category": "Architectural Wonder",
        "content": """
        <div style="max-width: 350px; font-family: Arial, sans-serif; line-height: 1.6;">
            <div style="background: #e74c3c; color: white; padding: 3px 8px; border-radius: 12px; 
                        font-size: 12px; display: inline-block; margin-bottom: 10px;">Architectural Wonder</div>
            
            <h3 style="color: #2c3e50; margin-top: 0; border-bottom: 2px solid #3498db; padding-bottom: 10px;">
                Reaching for the Sky: An American Dream in Steel
            </h3>
            
            <p>The Empire State Building stands as one of the most enduring symbols of human ambition and American 
            ingenuity. Completed in 1931 during the depths of the Great Depression, this 102-story Art Deco skyscraper 
            was built in just 410 days, a feat of construction that remains remarkable by today's standards.</p>
            
            <p>The building's construction began during the economic boom of the late 1920s, initiated by John J. Raskob, 
            former General Motors executive. The project was conceived as part of a fierce competition for height supremacy 
            in New York's skyline, racing against the Chrysler Building to claim the title of world's tallest structure.</p>
            
            <p>Architecturally, the Empire State Building represents the pinnacle of Art Deco design. Its setback design, 
            required by New York's 1916 Zoning Resolution, creates the distinctive silhouette that has become synonymous 
            with the New York skyline. The limestone facade creates vertical lines that emphasize the building's incredible height.</p>
            
            <p>The building's construction provided employment for over 3,500 workers during the Depression, becoming a symbol 
            of American resilience during economic hardship. The speed of construction was legendary - at its peak, workers 
            were completing 14 floors per week through innovative construction techniques.</p>
            
            <p>Pop culture has immortalized the Empire State Building through countless films, most notably the 1933 classic 
            "King Kong," where the giant ape made his final stand atop the building's spire. The building's famous lighting 
            system, installed in 1976, has become an integral part of New York's identity, changing colors to commemorate 
            holidays and significant events.</p>
        </div>
        """
    }
]

# Create base map
m = folium.Map(
    location=[40.7128, -74.0060],
    zoom_start=10,
    tiles='OpenStreetMap'
)

# Custom CSS and JavaScript for sidebar functionality
sidebar_css = """
<style>
    .sidebar {
        position: fixed;
        top: 0;
        left: 0;
        width: 400px;
        height: 100vh;
        background: white;
        box-shadow: 2px 0 10px rgba(0,0,0,0.1);
        z-index: 1000;
        transform: translateX(-100%);
        transition: transform 0.3s ease;
        overflow-y: auto;
        border-right: 1px solid #ddd;
    }
    
    .sidebar.open {
        transform: translateX(0);
    }
    
    .sidebar-header {
        background: #2c3e50;
        color: white;
        padding: 20px;
        position: relative;
        top: 0;
        z-index: 1001;
    }
    
    .sidebar-header h2 {
        margin: 0;
        font-size: 1.3em;
        padding-right: 40px;
    }
    
    .close-btn {
        position: absolute;
        right: 15px;
        top: 15px;
        background: none;
        border: none;
        color: white;
        font-size: 24px;
        cursor: pointer;
        padding: 5px;
        border-radius: 3px;
    }
    
    .close-btn:hover {
        background: rgba(255,255,255,0.2);
    }
    
    .sidebar-content {
        padding: 20px;
    }
    
    .custom-marker {
        background: #3498db;
        width: 35px;
        height: 35px;
        border-radius: 50%;
        border: 3px solid white;
        box-shadow: 0 2px 8px rgba(0,0,0,0.3);
        display: flex;
        align-items: center;
        justify-content: center;
        color: white;
        font-weight: bold;
        font-size: 14px;
        cursor: pointer;
        transition: transform 0.2s ease;
    }
    
    .custom-marker:hover {
        transform: scale(1.1);
    }
</style>
"""

sidebar_js = """
<script>
    function openSidebar(title, content) {
        document.getElementById('sidebar-title').innerHTML = title;
        document.getElementById('sidebar-content').innerHTML = content;
        document.getElementById('sidebar').classList.add('open');
    }
    
    function closeSidebar() {
        document.getElementById('sidebar').classList.remove('open');
    }
    
    // Close sidebar when clicking on map
    document.addEventListener('DOMContentLoaded', function() {
        var mapDiv = document.querySelector('.folium-map');
        if (mapDiv) {
            mapDiv.addEventListener('click', function(e) {
                if (e.target === mapDiv || e.target.classList.contains('leaflet-container')) {
                    closeSidebar();
                }
            });
        }
    });
</script>
"""

# Add sidebar HTML structure
sidebar_html = """
<div class="sidebar" id="sidebar">
    <div class="sidebar-header">
        <h2 id="sidebar-title">Location Details</h2>
        <button class="close-btn" onclick="closeSidebar()">&times;</button>
    </div>
    <div class="sidebar-content" id="sidebar-content">
        Select a marker to view details
    </div>
</div>
"""

# Add CSS, JavaScript, and HTML to the map
m.get_root().html.add_child(folium.Element(sidebar_css))
m.get_root().html.add_child(folium.Element(sidebar_js))
m.get_root().html.add_child(folium.Element(sidebar_html))

# Add markers to the map
for i, location in enumerate(locations, 1):
    # Create custom HTML for marker icon
    marker_html = f"""
    <div class="custom-marker" onclick="openSidebar('{location['title']}', `{location['content']}`)">
        {i}
    </div>
    """
    
    # Add marker with custom icon
    folium.Marker(
        location=location["coords"],
        icon=folium.DivIcon(
            html=marker_html,
            icon_size=(35, 35),
            icon_anchor=(17, 17)
        )
    ).add_to(m)

# Alternative approach using Popup (simpler but less customizable)
def create_with_popups():
    """Alternative approach using folium popups instead of sidebar"""
    m_popup = folium.Map(
        location=[40.7128, -74.0060],
        zoom_start=10,
        tiles='OpenStreetMap'
    )
    
    for i, location in enumerate(locations, 1):
        folium.Marker(
            location=location["coords"],
            popup=folium.Popup(
                html=location["content"],
                max_width=400,
                max_height=300
            ),
            tooltip=location["title"],
            icon=folium.Icon(
                color='blue',
                icon='info-sign',
                prefix='glyphicon'
            )
        ).add_to(m_popup)
    
    return m_popup

# Save the main map
m.save('interactive_map_with_sidebar.html')

# Save alternative popup version
m_popup = create_with_popups()
m_popup.save('interactive_map_with_popups.html')

print("Maps created successfully!")
print("1. interactive_map_with_sidebar.html - Main version with sidebar")
print("2. interactive_map_with_popups.html - Alternative with popups")

In [22]:
import folium
from folium import plugins

# Location data with detailed descriptions
locations = [
    {
        "coords": [40.7589, -73.9851],
        "title": "Times Square - The Crossroads of the World",
        "location": "New York City, NY",
        "category": "Tourist Destination",
        "content": """
        <div style="max-width: 350px; font-family: Arial, sans-serif; line-height: 1.6;">
            <div style="background: #e74c3c; color: white; padding: 3px 8px; border-radius: 12px; 
                        font-size: 12px; display: inline-block; margin-bottom: 10px;">Tourist Destination</div>
            
            <h3 style="color: #2c3e50; margin-top: 0; border-bottom: 2px solid #3498db; padding-bottom: 10px;">
                The Vibrant Heart of Manhattan
            </h3>
            
            <p>Times Square stands as one of the world's most visited tourist destinations, attracting over 
            50 million visitors annually. This bustling commercial intersection has evolved from a seedy area 
            in the 1970s to become the gleaming symbol of New York City's renaissance.</p>
            
            <p>The area gets its name from The New York Times, which moved its headquarters here in 1904. 
            What started as Longacre Square was transformed into Times Square, and with it came the tradition 
            of the New Year's Eve Ball Drop that has captivated millions worldwide for over a century.</p>
            
            <p>Today, Times Square is defined by its towering digital billboards, Broadway theaters, and 
            constant flow of pedestrians from every corner of the globe. The electronic displays consume 
            enough electricity to power a small city, yet they create an atmosphere unlike anywhere else on Earth.</p>
            
            <p>The economic impact of Times Square extends far beyond its physical boundaries. The area generates 
            billions in economic activity annually, supporting countless jobs in entertainment, retail, hospitality, 
            and media. Major corporations pay millions for prime advertising space.</p>
            
            <p>From a cultural perspective, Times Square represents the democratization of urban space. Here, 
            street performers share the same stage as multi-million dollar productions, and tourists from rural 
            communities experience the same sensory overload as seasoned city dwellers.</p>
        </div>
        """
    },
    {
        "coords": [40.6892, -74.0445],
        "title": "Statue of Liberty - Symbol of Freedom",
        "location": "Liberty Island, NY",
        "category": "Historic Monument",
        "content": """
        <div style="max-width: 350px; font-family: Arial, sans-serif; line-height: 1.6;">
            <div style="background: #e74c3c; color: white; padding: 3px 8px; border-radius: 12px; 
                        font-size: 12px; display: inline-block; margin-bottom: 10px;">Historic Monument</div>
            
            <h3 style="color: #2c3e50; margin-top: 0; border-bottom: 2px solid #3498db; padding-bottom: 10px;">
                Lady Liberty: A Gift That Became an Icon
            </h3>
            
            <p>The Statue of Liberty, officially known as "Liberty Enlightening the World," stands as perhaps 
            the most recognizable symbol of freedom and democracy in the modern world. This colossal neoclassical 
            sculpture was a gift from France to the United States in 1886.</p>
            
            <p>Designed by French sculptor Frédéric Auguste Bartholdi and engineered by Gustave Eiffel, the statue 
            represents Libertas, the Roman goddess of liberty. She holds a torch above her head with her right hand, 
            while her left hand carries a tablet inscribed with the date of the American Declaration of Independence.</p>
            
            <p>The statue's construction was a marvel of 19th-century engineering. Standing 305 feet tall from base 
            to torch, Lady Liberty was the tallest iron structure of its time. The copper exterior, which has developed 
            its distinctive green patina over the decades, was revolutionary in its construction technique.</p>
            
            <p>For millions of immigrants arriving in New York Harbor between 1886 and 1924, the Statue of Liberty 
            was the first glimpse of their new homeland. The famous poem "The New Colossus" by Emma Lazarus captures 
            this sentiment with its immortal words about welcoming the tired and poor.</p>
            
            <p>During both World Wars, the statue took on additional significance as a symbol of the democratic values 
            that Allied forces were fighting to protect. Today, over 4 million people visit Liberty Island annually, 
            making it one of the most popular tourist destinations in the United States.</p>
        </div>
        """
    },
    {
        "coords": [40.7505, -73.9934],
        "title": "Empire State Building - Art Deco Marvel",
        "location": "Midtown Manhattan, NY",
        "category": "Architectural Wonder",
        "content": """
        <div style="max-width: 350px; font-family: Arial, sans-serif; line-height: 1.6;">
            <div style="background: #e74c3c; color: white; padding: 3px 8px; border-radius: 12px; 
                        font-size: 12px; display: inline-block; margin-bottom: 10px;">Architectural Wonder</div>
            
            <h3 style="color: #2c3e50; margin-top: 0; border-bottom: 2px solid #3498db; padding-bottom: 10px;">
                Reaching for the Sky: An American Dream in Steel
            </h3>
            
            <p>The Empire State Building stands as one of the most enduring symbols of human ambition and American 
            ingenuity. Completed in 1931 during the depths of the Great Depression, this 102-story Art Deco skyscraper 
            was built in just 410 days, a feat of construction that remains remarkable by today's standards.</p>
            
            <p>The building's construction began during the economic boom of the late 1920s, initiated by John J. Raskob, 
            former General Motors executive. The project was conceived as part of a fierce competition for height supremacy 
            in New York's skyline, racing against the Chrysler Building to claim the title of world's tallest structure.</p>
            
            <p>Architecturally, the Empire State Building represents the pinnacle of Art Deco design. Its setback design, 
            required by New York's 1916 Zoning Resolution, creates the distinctive silhouette that has become synonymous 
            with the New York skyline. The limestone facade creates vertical lines that emphasize the building's incredible height.</p>
            
            <p>The building's construction provided employment for over 3,500 workers during the Depression, becoming a symbol 
            of American resilience during economic hardship. The speed of construction was legendary - at its peak, workers 
            were completing 14 floors per week through innovative construction techniques.</p>
            
            <p>Pop culture has immortalized the Empire State Building through countless films, most notably the 1933 classic 
            "King Kong," where the giant ape made his final stand atop the building's spire. The building's famous lighting 
            system, installed in 1976, has become an integral part of New York's identity, changing colors to commemorate 
            holidays and significant events.</p>
        </div>
        """
    }
]

# Custom CSS and JavaScript for sidebar functionality
sidebar_css = """
<style>
    .sidebar {
        position: fixed;
        top: 0;
        left: 0;
        width: 400px;
        height: 100vh;
        background: white;
        box-shadow: 2px 0 10px rgba(0,0,0,0.1);
        z-index: 1000;
        transform: translateX(-100%);
        transition: transform 0.3s ease;
        overflow-y: auto;
        border-right: 1px solid #ddd;
    }
    
    .sidebar.open {
        transform: translateX(0);
    }
    
    .sidebar-header {
        background: #2c3e50;
        color: white;
        padding: 20px;
        position: relative;
        top: 0;
        z-index: 1001;
    }
    
    .sidebar-header h2 {
        margin: 0;
        font-size: 1.3em;
        padding-right: 40px;
    }
    
    .close-btn {
        position: absolute;
        right: 15px;
        top: 15px;
        background: none;
        border: none;
        color: white;
        font-size: 24px;
        cursor: pointer;
        padding: 5px;
        border-radius: 3px;
    }
    
    .close-btn:hover {
        background: rgba(255,255,255,0.2);
    }
    
    .sidebar-content {
        padding: 20px;
    }
    
    .custom-marker {
        background: #3498db;
        width: 35px;
        height: 35px;
        border-radius: 50%;
        border: 3px solid white;
        box-shadow: 0 2px 8px rgba(0,0,0,0.3);
        display: flex;
        align-items: center;
        justify-content: center;
        color: white;
        font-weight: bold;
        font-size: 14px;
        cursor: pointer;
        transition: transform 0.2s ease;
    }
    
    .custom-marker:hover {
        transform: scale(1.1);
    }
</style>
"""

sidebar_js = """
<script>
    function openSidebar(title, content) {
        document.getElementById('sidebar-title').innerHTML = title;
        document.getElementById('sidebar-content').innerHTML = content;
        document.getElementById('sidebar').classList.add('open');
    }
    
    function closeSidebar() {
        document.getElementById('sidebar').classList.remove('open');
    }
    
    // Close sidebar when clicking on map
    document.addEventListener('DOMContentLoaded', function() {
        var mapDiv = document.querySelector('.folium-map');
        if (mapDiv) {
            mapDiv.addEventListener('click', function(e) {
                if (e.target === mapDiv || e.target.classList.contains('leaflet-container')) {
                    closeSidebar();
                }
            });
        }
    });
</script>
"""

# Add sidebar HTML structure
sidebar_html = """
<div class="sidebar" id="sidebar">
    <div class="sidebar-header">
        <h2 id="sidebar-title">Location Details</h2>
        <button class="close-btn" onclick="closeSidebar()">&times;</button>
    </div>
    <div class="sidebar-content" id="sidebar-content">
        Select a marker to view details
    </div>
</div>
"""


# Alternative approach using Popup (simpler but less customizable)
def create_with_popups():
    """Alternative approach using folium popups instead of sidebar"""
    m_popup = folium.Map(
        location=[40.7128, -74.0060],
        zoom_start=10,
        tiles='OpenStreetMap'
    )
    
    for i, location in enumerate(locations, 1):
        folium.Marker(
            location=location["coords"],
            popup=folium.Popup(
                html=location["content"],
                max_width=400,
                max_height=300
            ),
            tooltip=location["title"],
            icon=folium.Icon(
                color='blue',
                icon='info-sign',
                prefix='glyphicon'
            )
        ).add_to(m_popup)
    
    return m_popup


# Save alternative popup version
m_popup = create_with_popups()
m_popup.save('interactive_map_with_popups.html')

print("Maps created successfully!")
print("2. interactive_map_with_popups.html - Alternative with popups")

Maps created successfully!
2. interactive_map_with_popups.html - Alternative with popups
